# <center> STATS 607 - LECTURE 8
## <center> 10/01/2018

# <center> Numpy (Continuation)

Thank you Prof. Kerby Shedden for making available part of the material for this lecture.

## Arithmetic operations

As seen in the last lecture, Numpy arrays behave like mathematical vectors and matrices with respect to arithmetic operations. Lets see an example with a two dimensional Numpy array:

In [1]:
import numpy as np

x = np.ones((3,4)) # Creates a two dimensional numpy array.
x, x.dtype

(array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]), dtype('float64'))

In [2]:
print(x.ndim)  # Returns number of dimensions in an array.
print(x.shape) # Return tuple desribing array shape.
print(x.size)  # Returns number of elements.

2
(3, 4)
12


In [3]:
y = np.array([[1, 2], [3, 4]])
z = np.array([[5, 6], [7, 8]])

In [4]:
print(y)
print(z)

[[1 2]
 [3 4]]
[[5 6]
 [7 8]]


In [5]:
y + z # Summs element wise.

array([[ 6,  8],
       [10, 12]])

In [6]:
np.round(y / z, 2) # Divides element wise.

array([[0.2 , 0.33],
       [0.43, 0.5 ]])

In [7]:
w = np.array([[1, 2], [3, 4], [5, 6]]) # Lets create a new Numpy array 3x2.
w

array([[1, 2],
       [3, 4],
       [5, 6]])

In [8]:
w + z # Notice the error message below. We'll get to it later in this lecture.

ValueError: operands could not be broadcast together with shapes (3,2) (2,2) 

An easy way to avoid making copies when performing array arithmetic in Numpy is to use the in-place arithmetic operators +=, *=, etc. When we use x = x + y, a new allocation is made to hold the value x + y, and this allocated memory is then assigned to x, with the previous memory block of x (eventually) being garbage collected. But x += y does not result in a new allocation, as seen below:

In [9]:
y = np.array([[1, 2], [3, 4]])
z = np.array([[5, 6], [7, 8]])

print(id(y))
y = y + z   # Regular sum.
print(id(y))
print(y)

4595897888
4595813552
[[ 6  8]
 [10 12]]


In [10]:
y = np.array([[1, 2], [3, 4]])
z = np.array([[5, 6], [7, 8]])

print(id(y))
y += z        # In-place sum.
print(id(y))
print(y)

4595813632
4595813632
[[ 6  8]
 [10 12]]


## Indexing and Slicing

In [11]:
x = np.array([[1, 2], [3, 4]])
x

array([[1, 2],
       [3, 4]])

In [12]:
print(x[0]) # Returns the first line.
print(x[1:]) # Returns a similar structure to x with the second line.
print(x[0][1]) # Returns the second element of the first line.
print(x[0,1]) # A more concise way of writing the above lookup.

[1 2]
[[3 4]]
2
2


Remember that slices will normally return a “view” of the underlying data, meaning that if you change a slice, the same values will change in the parent array:

In [13]:
x = np.array([[1, 2, 3], [4, 5, 6]])
y = x[:,0] # retrieves the first column.
y[1]=100 # modifies its element.
x

array([[  1,   2,   3],
       [100,   5,   6]])

Now suppose you want to retrieve elements from the first and the third columns of x:

In [14]:
x = np.array([[1, 2, 3], [4, 5, 6]])
y = x[:,[0,2]] # We use a list to index - this is also called advanced or fancy indexing.
y[1]=100
x

array([[1, 2, 3],
       [4, 5, 6]])

When you use advanced indexing, no view is provided and a copy of the original object will be made.

In [15]:
x = np.array([[1,2], [3,4], [5,6]])
y = x[[-1,-2],:] # Another example of advanced indexing with negative indices.
y

array([[5, 6],
       [3, 4]])

In [16]:
x = np.array([[1,-2], [3,4], [-5,6]])
print(x)
print(x<0) # x <0 is a boolean array.

[[ 1 -2]
 [ 3  4]
 [-5  6]]
[[False  True]
 [False False]
 [ True False]]


In [17]:
x[x<0]=0 # This is called boolean indexing. Here I am setting negative entries to zero.
x

array([[1, 0],
       [3, 4],
       [0, 6]])


## Linear Algebra

Numpy supports a wide range of matrix and linear algebra operations. For example, you can use 'np.linalg.solve' to solve a linear system of equations, 'np.linalg.svd' for the singular value decomposition, 'np.linalg.cholesky' for the cholesky decomposition, and much more... We will not attempt to describe all of them here. Just a couple of quick examples:

In [18]:
u = np.array([1,2])
v = np.array([1,2])

a = np.dot(u, v)   # inner product.
b = np.outer(u, v) # outer product.
print('Inner Product: \n', a)
print('Outer Product: \n', b)

Inner Product: 
 5
Outer Product: 
 [[1 2]
 [2 4]]


In [19]:
A = np.random.randint(0,10,(2,2))
A
[L,V] = np.linalg.eig(A)
print('Eigen Values: \n', L)
print('Eigen Vectors: \n', V)

Eigen Values: 
 [6. 0.]
Eigen Vectors: 
 [[ 1.  -0.8]
 [ 0.   0.6]]


## Vectorization

Vectorization within NumPy is used to express operations as occurring on entire arrays rather than their individual elements. Here’s a definition from Wes McKinney:

"
This practice of replacing explicit loops with array expressions is commonly referred to as vectorization. In general, vectorized array operations will often be one or two (or more) orders of magnitude faster than their pure Python equivalents, with the biggest impact seen in any kind of numerical computations. (see [here](https://www.safaribooksonline.com/library/view/python-for-data/9781449323592/ch04.html?orpq))
"

Lets see an example:

In [20]:
values = np.random.choice([True,False], size=500)

In [21]:
def count_transitions1(values):
    """Returns the number of transitions from either False to True or from True to False"""
    output = 0
    for x,y in zip(values[:-1],values[1:]):
        if x!=y:
            output+=1
    return output

In [22]:
def count_transitions2(values):
    """Returns the number of transitions from either False to True or from True to False"""
    output = np.sum(values[:-1]!=values[1:])
    return output

In [23]:
print(count_transitions1(values))
print(count_transitions2(values))

237
237


In [24]:
%timeit -n 1000 count_transitions1(values)
%timeit -n 1000 count_transitions2(values)

780 µs ± 11.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
5.5 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Reducing functions

Numpy has reducing functions that collapse a multidimensional array to one single axis. The axes are numbered 0 (rows), 1 (columns), etc. For example:

In [25]:
x = np.random.normal(size=(5,10))
x

array([[ 9.20961197e-01, -5.32602392e-01,  6.71148120e-01,
        -1.96888526e+00, -9.88423666e-02, -1.10787200e+00,
        -3.05209143e-01, -3.53075775e-01, -1.59356410e+00,
         6.84701668e-01],
       [ 6.73556695e-01, -1.78395194e-01,  2.98747481e-01,
         1.40250253e-02, -1.19106895e+00, -1.08183859e+00,
         1.19804015e+00,  6.68441672e-01,  5.28948071e-02,
         1.18164905e-01],
       [ 1.37286137e+00, -3.95800504e-03,  1.82497216e+00,
        -1.52650632e-01,  2.50971108e-01, -7.09316101e-01,
        -6.11018946e-01,  1.42337377e-01,  2.32216527e-01,
        -7.25618288e-02],
       [ 1.99726405e+00, -9.32739334e-02,  2.61412469e-01,
        -1.59183095e+00,  1.83075133e-01, -7.19270463e-01,
        -8.01663911e-01,  5.88805950e-01,  2.02972829e+00,
        -8.38174782e-01],
       [ 1.86767916e+00, -4.44231975e-01, -5.65877680e-01,
         1.20664431e+00,  1.07077203e+00,  6.72052668e-01,
        -3.37373987e-01, -6.94018342e-01, -2.01728966e-03,
         5.

In [26]:
print(x.mean(0)) # column-wise means, size=10.
print(x.mean(1)) # row-wise means, size=5.

[ 1.3664645  -0.2504923   0.49808051 -0.4985395   0.04298139 -0.5892489
 -0.17144517  0.07049818  0.14385165  0.0788001 ]
[-0.368324    0.0572568   0.2273853   0.10160719  0.32754994]


## Broadcasting

The term broadcasting describes how numpy treats arrays with different shapes during arithmetic operations. Subject to certain constraints, the smaller array is “broadcast” across the larger array so that they have compatible shapes. Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python.

One use of broadcasting is if we want to center or scale an array by column:

In [27]:
x = np.random.random(size=(10000, 4))
print(id(x))

print(x.mean(0))
print(x.std(0))

print(x.shape)
print(x.mean(0).shape)

x -= x.mean(0)
x /= x.std(0)

print(x.mean(0))
print(x.std(0))

print(id(x))

4596441536
[0.50250989 0.50371817 0.50473372 0.49721157]
[0.28761965 0.2907155  0.28583363 0.29019522]
(10000, 4)
(4,)
[-3.68691189e-15  1.50657264e-16 -6.66697808e-15  5.73615044e-15]
[1. 1. 1. 1.]
4596441536


In the example above, x.mean(0) returns an array with dimension (4,), which matches from the right with the dimension of x, which is (10000,4). Therefore the shapes are compatible for broadcasting. The behavior in this case is that the result of x.mean(0) is only computed one time, and the same result is used for centering each row of x.

There is a special case of the broadcasting rules that applies when a dimension’s length is equal to 1. In this case, the value in that dimension is copied to match the dimension on the same axis in the other array:

In [28]:
a = np.array([10.0, 15.0, 20.0])
b = np.array([5.0, 5.0, 5.0])
a / b

array([2., 3., 4.])

In [29]:
a = np.array([10.0, 15.0, 20.0])
print(a.shape)
b = np.array([5.0])
print(b.shape)
a / b

(3,)
(1,)


array([2., 3., 4.])

In [30]:
x = np.random.normal(size=(10, 2))
y = np.random.normal(size=(10, 1))
print(x + y)

[[-1.87910323 -1.95398868]
 [ 0.2889479   0.80727502]
 [ 0.35499204  0.71939663]
 [-1.87803902  0.04162347]
 [-0.61127833  0.78452084]
 [-0.07130332  0.40199372]
 [-1.24470526 -0.74698243]
 [-1.45926702  0.33395947]
 [ 1.0873452   0.44111197]
 [ 1.19793798  1.92453536]]


There is a special syntax for adding a new axis of length 1 to an array:

In [31]:
x = np.zeros(10)  # shape is (10,).
y = x[:, None]    # shape is (10,1).
z = x[None, :]    # shape is (1,10).

A common setting where this is useful is when you want to center or scale a two-dimensional array by row. Adding a new column with dimension 1 allows the broadcasting rules to apply when they otherwise would not:

In [32]:
x = np.random.normal(size=(10, 3))
x -= x.mean(1)[:, None]
x /= x.std(1)[:, None]

See the intermediate values below:

In [33]:
x.mean(1), x.mean(1).shape

(array([-7.40148683e-17,  3.70074342e-17, -7.40148683e-17,  7.40148683e-17,
         0.00000000e+00,  3.70074342e-17,  1.11022302e-16,  7.40148683e-17,
        -3.70074342e-17, -3.70074342e-17]), (10,))

In [34]:
x.mean(1)[:, None], x.mean(1)[:, None].shape

(array([[-7.40148683e-17],
        [ 3.70074342e-17],
        [-7.40148683e-17],
        [ 7.40148683e-17],
        [ 0.00000000e+00],
        [ 3.70074342e-17],
        [ 1.11022302e-16],
        [ 7.40148683e-17],
        [-3.70074342e-17],
        [-3.70074342e-17]]), (10, 1))

## Conclusion

Numpy is one of many tools designed from the 1980’s to the early 2000’s for array manipulation. These tools tend to follow a common set of design principles, namely:

* Arrays are contiguous in memory.
* Memory management is dynamic and mostly invisible.
* Mathematical operations are expressed in the same syntax that we usually use to write mathematics on paper, e.g. x = y + z assigns to x the pointwise sum of y and z.

These design choices favor ease-of-use, but there is a loss of performance in many cases, and may limit the ability to work on very large data objects. One consequence of this design strategy is that copies are sometimes made that are not needed. For example, consider the following:

In [ ]:
x = np.arange(10)
y = np.arange(10)
z = np.sum(x * y)

In creating z, the pointwise product x * y is formed as an array of length 10, and then is summed. It would be faster and more memory efficient to accumulate the sum in a loop, e.g. expressed in Python the loop would be:

In [ ]:
z = 0
for i in range(10):
    z += x[i] * y[i]

This particular operation exists in numpy via the dot function. We can compare the timings:

In [35]:
m = 1000
x = np.arange(m)
y = np.arange(m)

def f():
    return np.sum(x * y)

def g():
    return np.dot(x, y)

%timeit -n 10000 f()
%timeit -n 10000 g()

5.41 µs ± 509 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.59 µs ± 22.9 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


The challenge from a design perspective here is that the Python interpreter handles the product x * y and immediately allocates space for the result. There is no way for Numpy to analyze this algebraic expression and recognize that the product array is only going to be summed, and therefore need not be constructed in entirety.

There is now great interest in next-generation array processing tools that aim to resolve the performance limitations of Numpy. For example, to address the issue of excess copying, the 'numexpr' package takes expressions written as strings and evaluates them in a virtual machine that can apply a variety of accelerations. For example, by passing “sum(x * y)” to the virtual machine, it can be automatically be determined that the sum can be calculated in streaming fashion without explicitly forming x * y.

Other projects include Numba, which uses just-in-time compilation to bypass the Python interpreter (see [here](https://numba.pydata.org)); Cython, which uses an extended Python-like language to permit compilation of code to C [here](http://cython.org); Dask, which defines array-like data containers that can live in either primary memory or on-disk [here](http://docs.dask.org/en/latest/); and Theano, which generates code for doing array processing on GPUs (among other things) [here](http://deeplearning.net/software/theano/).